<a href="https://colab.research.google.com/github/L-Machado/UNIARAIA1/blob/main/RedesCNN_ReconhecimentoImagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Carregar o conjunto de dados MNIST (Imagens com Dados de 0 a 9)
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalizar os valores dos pixels para o intervalo [0, 1]
train_images, test_images = train_images / 255.0, test_images / 255.0

# Adicionar uma dimensão para os canais de cor (preto e branco neste caso)
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# Definir o modelo CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Treinnar o modelo
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

Epoch 1/5
938/938 [==============================] - 59s 62ms/step - loss: 0.1749 - accuracy: 0.9486 - val_loss: 0.0445 - val_accuracy: 0.9856
Epoch 2/5
938/938 [==============================] - 58s 62ms/step - loss: 0.0507 - accuracy: 0.9847 - val_loss: 0.0410 - val_accuracy: 0.9859
Epoch 3/5
938/938 [==============================] - 61s 65ms/step - loss: 0.0342 - accuracy: 0.9890 - val_loss: 0.0366 - val_accuracy: 0.9876
Epoch 4/5
938/938 [==============================] - 66s 71ms/step - loss: 0.0279 - accuracy: 0.9910 - val_loss: 0.0282 - val_accuracy: 0.9904
Epoch 5/5
938/938 [==============================] - 60s 64ms/step - loss: 0.0225 - accuracy: 0.9926 - val_loss: 0.0294 - val_accuracy: 0.9903


In [4]:
model.save('/content/modelo_mnist.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import requests
from io import BytesIO
from PIL import Image
model = tf.keras.models.load_model('/content/modelo_mnist.h5')

# URL da imagem
image_url = 'https://ae03.alicdn.com/kf/Hef69ad6bbde84bb3ae35b00569854206R.jpg_640x640q90.jpg'

# Função para baixar e processar a imagem
def download_and_process_image(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()  # Verifica se o download foi bem-sucedido
        img = Image.open(BytesIO(response.content))
        img = img.convert("L")  # Converte a imagem para escala de cinza
        img = img.resize((28, 28))  # Redimensiona a imagem para 28x28
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=-1)  # Adiciona uma dimensão para os canais de cor
        img_array = np.expand_dims(img_array, axis=0)  # Adiciona uma dimensão para o lote
        img_array /= 255.0  # Normalização

        return img_array
    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar a imagem: {e}")
    except UnidentifiedImageError:
        print(f"Erro ao identificar a imagem: {image_url}")
    except Exception as e:
        print(f"Erro ao processar a imagem: {e}")
    return None

# Processar a imagem
processed_image = download_and_process_image(image_url)

# Verificar se a imagem foi processada corretamente
if processed_image is not None:
    # Fazer a previsão
    prediction = model.predict(processed_image)
    predicted_class = np.argmax(prediction)

    # Mapear o número da classe para o nome da classe
    class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    predicted_label = class_names[predicted_class]

    print('Predicted class:', predicted_label)
else:
    print("Falha no processamento da imagem.")

1/1 [==============================] - 0s 81ms/step
Predicted class: 2
